In [0]:
from cni_connectors import adls_gen1_connector as adls_conn
var_adls_uri = adls_conn.adls_gen1_connect(spark, dbutils, scope="adls_gen2", dynamic_overwrite="dynamic")

fornecedor_dev
dev-execution


In [0]:
from unicodedata import normalize

import crawler.functions as cf
import json
import pyspark.sql.functions as f
import re
import os


In [0]:
table = json.loads(re.sub("\'", '\"', dbutils.widgets.get("tables")))
dls = json.loads(re.sub("\'", '\"', dbutils.widgets.get("dls")))
adf = json.loads(re.sub("\'", '\"', dbutils.widgets.get("adf")))

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-1878055890524620>:1
----> 1 table = json.loads(re.sub("\'", '\"', dbutils.widgets.get("tables")))
      2 dls = json.loads(re.sub("\'", '\"', dbutils.widgets.get("dls")))
      3 adf = json.loads(re.sub("\'", '\"', dbutils.widgets.get("adf")))

File /databricks/python_shell/dbruntime/WidgetHandlerImpl.py:42, in WidgetsHandlerImpl.get(self, name)
     36 def get(self, name):
     37     """ Returns the current value of a widget with give name.
     38 
     39     :param name: Name of the argument to be accessed
     40     :return: Current value of the widget or default value
     41     """
---> 42     return self._notebookArguments.getArgument(name, self._entry_point.getCurrentBindings())

File /databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py:1321, in JavaMember.__call__(self, *args)
   1315 comman

In [0]:
table = {"schema":"ibge","table":"pintec_n_grau_obst_cnae","prm_path":"usr/ibge_pintec/FIEC_IBGE_pintec_n_grau_obst_cnae_mapeamento_unificado_para_correcao.xlsx",
          "destination":"crw/oni/ibge/pintec/n_grau_obst/n_grau_obst_cnae/"}

dls = {"folders":{"landing":"/tmp/dev/lnd","error":"/tmp/dev/err","archive":"/tmp/dev/ach","staging":"/tmp/dev/stg","log":"/tmp/dev/log","raw":"/tmp/dev/raw","trusted":"/tmp/dev/trs","business":"/tmp/dev/biz","prm":"/tmp/dev/prm","historico":"/tmp/dev/hst"},"path_prefix":"tmp","uld":{"folders":{"landing":"/tmp/dev/uld","error":"/tmp/dev/err","staging":"/tmp/dev/stg","log":"/tmp/dev/log","raw":"/tmp/dev/raw","archive":"/tmp/dev/ach"},"systems":{"raw":"usr"},"path_prefix":"/tmp/dev/"},"systems":{"raw":"usr"}}

adf = {"adf_factory_name":"cnibigdatafactory","adf_pipeline_name":"trs_biz_rfb_cno","adf_pipeline_run_id":"c158e9dd-98df-4f7b-a64d-d13c76669868","adf_trigger_id":"67c514a7245449b984eb4aadd55bfbff","adf_trigger_name":"Sandbox","adf_trigger_time":"2023-08-22T21:22:42.5769844Z","adf_trigger_type":"Manual"}
 

In [0]:
lnd = dls['folders']['landing']
raw = dls['folders']['raw']

lnd_path = "{lnd}/crw/{schema}__{table}".format(lnd=lnd, schema=table["schema"], table=table["table"])

In [0]:
raw_path = "{raw}/{dst}".format(raw=raw, dst=table["destination"])

adl_sink = "{adl_path}{raw_path}".format(adl_path=var_adls_uri, raw_path=raw_path)

In [0]:
print(raw_path)

/tmp/dev/raw/crw/oni/ibge/pintec/n_grau_obst/n_grau_obst_cnae/


In [0]:
def __normalize_str(_str):
    return re.sub(r'[,;{}()\n\t=-]', '', normalize('NFKD', _str)
                  .encode('ASCII', 'ignore')
                  .decode('ASCII')
                  .replace(' ', '_')
                  .replace('-', '_')
                  .upper())

In [0]:
prm_path = os.path.join(dls['folders']['prm'], table["prm_path"])

In [0]:
headers = {'name_header':'Campo Origem','pos_header':'C','pos_org':'C','pos_dst':'E','pos_type':'F'}
var_prm_dict = cf.parse_ba_doc(dbutils, prm_path, headers=headers)

In [0]:
for sub_plan in var_prm_dict.values():
  for row in sub_plan:
    row[0] = __normalize_str(row[0])

In [0]:
def __transform_columns(sheet: str):
  for org, dst, _type in var_prm_dict[sheet]:
    if org == 'N/A':
      yield f.lit(None).cast(_type).alias(dst)
    else:
      yield f.col(org).cast(_type).alias(dst)

In [0]:
_df = None
for lnd_year_path in cf.list_subdirectory(dbutils, lnd_path):
  df_adl_files = cf.list_adl_files(spark, dbutils, lnd_year_path)

  files_folder = cf.list_subdirectory(dbutils, lnd_year_path)

  for i, file in enumerate(files_folder):
    path_origin = '{adl_path}/{file_path}'.format(adl_path=var_adls_uri, file_path=file)
    year = lnd_year_path.split('/')[-1].replace('.parquet', '').split('_')[-1]
    df = spark.read.parquet(path_origin)

    cf.check_ba_doc(df, parse_ba=var_prm_dict, sheet=year)

    df = df.select(*__transform_columns(year))
    df = df.withColumn("ANO", f.lit(year))
    
    _df_n_columns = len(_df.columns) if _df else len(df.columns)

    if len(df.columns) != _df_n_columns:
      biggest_df = df if len(df.columns) > len(_df.columns) else _df
      smallest_df = df if len(df.columns) > len(_df.columns) else _df      
    else:
      biggest_df = _df
      smallest_df = df
      
    if _df:
      _df = biggest_df.unionByName(smallest_df)      
    else:
      _df = df
      
  if _df:
    df = _df

df = cf.append_control_columns(df, dh_insercao_raw=adf["adf_trigger_time"].split(".")[0])
df = df.join(f.broadcast(df_adl_files), on='nm_arq_in', how='inner')

In [0]:
df.display()

nm_arq_in,NM_ATIVIDADES_DA_INDUSTRIA,NR_EMPRESAS_TOTAL,NR_N_IMPLEMENT_INOV_PROD_PROC_NEG_TOTAL,NR_DIFICUL_OBST_TOTAL,NR_GRAU_INST_ECO_IMP,NR_GRAU_INST_ECO_POU_IMP,NR_GRAU_INST_ECO_N_IMP,NR_GRAU_ACIR_CONC_IMP,NR_GRAU_ACIR_CONC_POU_IMP,NR_GRAU_ACIR_CONC_N_IMP,NR_GRAU_BAIX_DEM_IMP,NR_GRAU_BAIX_DEM_POU_IMP,NR_GRAU_BAIX_DEM_N_IMP,NR_GRAU_DIF_PARC_IMP,NR_GRAU_DIF_PARC_POU_IMP,NR_GRAU_DIF_PARC_N_IMP,NR_GRAU_LIM_REC_INT_IMP,NR_GRAU_LIM_REC_INT_POU_IMP,NR_GRAU_LIM_REC_INT_N_IMP,NR_GRAU_LIM_TEC_EXT_IMP,NR_GRAU_LIM_TEC_EXT_POUC_IMP,NR_GRAU_LIM_TEC_EXT_N_IMP,NR_GRAU_ESC_REC_PUB_IMP,NR_GRAU_ESC_REC_PUB_POU_IMP,NR_GRAU_ESC_REC_PUB_N_IMP,NR_GRAU_MUD_PRIO_EST_IMP,NR_GRAU_MUD_PRIO_EST_POU_MP,NR_GRAU_MUD_PRIO_EST_N_IMP,NM_ARQUIVO,ANO,nr_reg,dh_insercao_raw,dh_arq_in
PINTEC_N_GRAU_OBST_CNAE_2021.parquet,Total Indústria,9400,2722,921,805,47,68,438,362,121,540,255,125,332,307,282,480,320,120,299,359,262,475,312,134,404,274,242,2021_PINTEC_N_GRAU_OBST_CNAE,2021,0,2023-08-22T21:22:42.000+0000,2023-08-10T16:15:28.000+0000
PINTEC_N_GRAU_OBST_CNAE_2021.parquet,Indústrias extrativas,240,103,13,13,0,0,0,13,0,11,0,2,0,2,11,11,2,0,0,13,0,2,0,11,2,11,0,2021_PINTEC_N_GRAU_OBST_CNAE,2021,1,2023-08-22T21:22:42.000+0000,2023-08-10T16:15:28.000+0000
PINTEC_N_GRAU_OBST_CNAE_2021.parquet,Indústrias de transformação,9160,2618,908,792,47,68,438,349,121,529,255,123,332,305,271,469,318,120,299,346,262,473,312,122,402,263,242,2021_PINTEC_N_GRAU_OBST_CNAE,2021,2,2023-08-22T21:22:42.000+0000,2023-08-10T16:15:28.000+0000
PINTEC_N_GRAU_OBST_CNAE_2021.parquet,Fabricação de produtos alimentícios,1613,633,220,164,0,55,138,81,0,136,83,0,109,83,27,163,56,0,136,28,54,135,84,0,135,56,27,2021_PINTEC_N_GRAU_OBST_CNAE,2021,3,2023-08-22T21:22:42.000+0000,2023-08-10T16:15:28.000+0000
PINTEC_N_GRAU_OBST_CNAE_2021.parquet,Fabricação de bebidas,143,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021_PINTEC_N_GRAU_OBST_CNAE,2021,4,2023-08-22T21:22:42.000+0000,2023-08-10T16:15:28.000+0000
PINTEC_N_GRAU_OBST_CNAE_2021.parquet,Fabricação de produtos do fumo,25,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021_PINTEC_N_GRAU_OBST_CNAE,2021,5,2023-08-22T21:22:42.000+0000,2023-08-10T16:15:28.000+0000
PINTEC_N_GRAU_OBST_CNAE_2021.parquet,Fabricação de produtos têxteis,436,84,63,63,0,0,25,22,15,23,17,22,22,2,38,22,39,1,2,22,38,24,16,22,17,23,22,2021_PINTEC_N_GRAU_OBST_CNAE,2021,6,2023-08-22T21:22:42.000+0000,2023-08-10T16:15:28.000+0000
PINTEC_N_GRAU_OBST_CNAE_2021.parquet,Confecção de artigos do vestuário e acessórios,568,210,108,84,23,0,25,28,55,29,24,55,24,29,55,26,27,55,2,51,55,27,26,55,27,2,79,2021_PINTEC_N_GRAU_OBST_CNAE,2021,7,2023-08-22T21:22:42.000+0000,2023-08-10T16:15:28.000+0000
PINTEC_N_GRAU_OBST_CNAE_2021.parquet,"Preparação de couros e fabricação de artefatos de couro, artigos de viagem e calçados",343,108,62,62,0,0,42,0,19,41,19,1,41,20,0,62,0,0,19,41,1,40,20,1,41,20,1,2021_PINTEC_N_GRAU_OBST_CNAE,2021,8,2023-08-22T21:22:42.000+0000,2023-08-10T16:15:28.000+0000
PINTEC_N_GRAU_OBST_CNAE_2021.parquet,Fabricação de produtos de madeira,230,132,19,10,9,0,10,9,0,9,1,9,10,0,9,18,1,0,9,9,1,19,0,0,9,1,9,2021_PINTEC_N_GRAU_OBST_CNAE,2021,9,2023-08-22T21:22:42.000+0000,2023-08-10T16:15:28.000+0000


In [0]:
df.write.parquet(path=adl_sink, mode='overwrite')